In [1]:
#from gurobipy import *

# Get some additional dependencies
import pandas as pd
import numpy as np
import random
import math

ImportError: No module named 'pandas'

In [5]:
# Read the dataset
data = pd.read_csv('../data.csv')
data.replace('-', 1, inplace = True)
city_names = list(data)[1:]

# Denote model variables
N = len(data)
M = len(data)
DC = [i for i in range(0, N)]
DA = [i for i in range(0, M)]

# Hold a mapping from indexes to city names for showing results later
cities = dict(zip(DC, city_names))

# Get input data as a distance matrix for fast random access
distance = data.as_matrix()[:,1:].astype(int)

# DCs and DAs have no ordering, therefore I will represent them as sets
DC = set(DC)
DA = set(DA)

In [2]:
# Hyper parameters - these will never change in this assignment
FIXED_DEMAND = 100
FIXED_CAPACITY = len(city_names) * FIXED_DEMAND
FIXED_COST = 100000
NUM_OPENED = 7

S = FIXED_CAPACITY * np.ones(N)
D = FIXED_DEMAND * np.ones(M)

NameError: name 'city_names' is not defined

In [20]:
def nearest_neighbor(here, world):
    min_distance = 10000
    closest = -1
    for place in world:
        if(distance[here, place] < min_distance):
            min_distance = distance[here, place]
            closest = place
    return min_distance, closest
        
    

def compute_cost(opened):
    """Given a set of opened DCs, computes the total transportation costs to satisfy all DAs"""    
    cost = 0
    for da in DA:
        dist, _ = nearest_neighbor(da, opened)
        cost += dist     
    return cost

def update_temp(T, rep):
    re
    turn T/math.log(rep + 1)

def maybe_mutate(opened, T):
    """
    Mutates a solution to a random neighbor. Note: this is not referentially transparent and therefore
    not thread safe.
    """
    def get_neighbor():
        check = opened.copy()
        previous = random.sample(opened, 1)[0]
        _, new = nearest_neighbor(previous, DC.difference(opened))
        check.remove(previous)
        check.add(new)
        return check
    

    original_cost = compute_cost(opened)
    neighbor = get_neighbor()
    new_cost = compute_cost(neighbor)
    delta = new_cost - original_cost
    
    # Should we make the transition?
    p = random.random();
    if(p <= math.exp((-1*delta)/(T*1.0)) or delta < 0):
        return neighbor
    return opened

def simulated_annealing(start_T = 1000, stop_T = 0.001, max_reps = 40000):
    print("Starting annealing....")
    # Get an initial solution
    opened = set(random.sample(DC, NUM_OPENED))
    
    # Best solution found globally
    optimum = opened.copy()
    optimal_cost = compute_cost(optimum)
    
    rep = 1
    T = start_T
    
    while(T > stop_T and rep < max_reps):
        opened = maybe_mutate(opened, T)
        if(compute_cost(opened) < optimal_cost):
            optimum = opened.copy()
            optimal_cost = compute_cost(opened)
            
        T = update_temp(start_T, rep)
        rep += 1
    
    print("Best solution found: " + str(optimum) + 
          "\nwith total transportation costs: " + str(optimal_cost))
    

We can now evaluate the convergence of our heuristic since we now the global optimum solution from
the LP implementation (see assignment 2). The optimal solution is opening the following DCs:

* Athens   -   **2**
* Madrid   -  **20**
* Naples   -  **24**
* Turin    -  **33**
* Munich   -  **23**
* Hamburg  -  **14**
* Brussels -  ** 6**

This solution gives us the minimum achievable cost of **L = 9855**

In [21]:
simulated_annealing()


Starting annealing....
Best solution found: {5, 21, 22, 8, 13, 14, 31}
with total transportation costs: 15451


The results are nowhere close the global optimum, however depending on our specific needs they could be considered "good enough". Since this is a stochastic heuristic, its results are not reproducible (every run will yield a different solution). However a sample result in acceptable runtime is the set:

**{1, 17, 20, 5, 28, 13, 31}** yielding a cost **L = 12165**

